In [4]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(
    temperature=0.1
)



In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.prompts import PromptTemplate, ChatPromptTemplate

template = PromptTemplate.from_template("")

chat = ChatOpenAI(
    temperature=0.1
)

messages = [
    SystemMessage(content="Your are a geography expert, Ant you only reply in {language}"),
    AIMessage(content="Ciao, mi chiamo {name}"),
    HumanMessage(content="What is the distance between {country_a} and {country_b}. Also, what is your name?")
]

chat.predict_messages(messages)

AIMessage(content='Mi dispiace, non posso fornire informazioni sulla distanza tra due paesi in questa lingua. Il mio nome è Ant. Come posso aiutarti con la geografia?')

In [2]:
#Prompt Templates

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)
# template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}")

# prompt = template.format(country_a = "Mexico", country_b = "Thailand")
# chat.predict(prompt)

template = ChatPromptTemplate.from_messages([
    ("system", "Your are a geography expert, Ant you only reply in {language}"),
    ("ai", "Ciao, mi chiamo {name}"),
    ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?")
])

prompt = template.format_messages(
    language="Greek",
    name="Socrates",
    country_a ="Mexico",
    country_b="Thailand",
)

chat.predict_messages(prompt)


AIMessage(content='Γεια σας! Το όνομά μου είναι Σωκράτης. Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα.')

In [2]:
# OutputParser and LCEL
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import BaseOutputParser

chat = ChatOpenAI(temperature=0.1)

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items= text.strip().split(",")
        return list(map(str.strip, items))


template = ChatPromptTemplate.from_messages([
    ("system", "Your are a list generathing machine. Everything you are asked will be answered with a comma seperated list of max {max_items}. Do NOT reply with anything else."),
    ("human", "{question}")
])

chain = template | chat | CommaOutputParser()

chain.invoke({
    "max_items": 10,
    "question":"What are the planets",
})




['Mercury',
 'Venus',
 'Earth',
 'Mars',
 'Jupiter',
 'Saturn',
 'Uranus',
 'Neptune',
 'Pluto']